# Анализ популярности музыки: Billboard и UK Charts

Этот проект анализирует данные о популярности музыкальных треков из двух источников:
- Billboard Charts (США)
- UK Top 100 Charts (Великобритания)

## Этапы анализа:
1. Загрузка и подготовка данных
2. Очистка и нормализация
3. Объединение данных из разных источников
4. Получение аудио-файлов через YouTube
5. Анализ музыкальных характеристик

## Используемые библиотеки
- pandas, numpy: для обработки данных
- matplotlib, seaborn: для визуализации
- yt-dlp: для загрузки аудио с YouTube
- librosa: для анализа аудио
- torch: для машинного обучения

In [ ]:
# Базовые библиотеки для анализа данных
%pip install pandas numpy matplotlib seaborn
%pip install openpyxl pyarrow

# Библиотеки для работы с YouTube и аудио
%pip install yt-dlp youtube-search-python
%pip install httpx==0.24.1
%pip install librosa
%pip install spotipy

# Библиотеки для машинного обучения
%pip install torch torchvision
%pip install scikit-learn
%pip install tensorflow
%pip install keras

# Библиотеки для работы со Spotify API
%pip install python-dotenv spotipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
'''
# Load credentials from .env securely
from dotenv import load_dotenv
import os
load_dotenv()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Read credentials from environment (.env is gitignored)
client_id = os.environ.get('SPOTIFY_CLIENT_ID')
client_secret = os.environ.get('SPOTIFY_CLIENT_SECRET')
if not client_id or not client_secret:
    raise RuntimeError('SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET must be set in .env or environment')

# Create auth manager using env credentials
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Далее работаем с файлами
import pandas as pd

df = pd.read_csv('songs_for_spotify.csv')  # файл с колонками title, artist

# Подготовим поле для Spotify ID
df['spotify_id'] = None
# df = объединённый Billboard+UK (или только UK) после чистки
uniq = (df[['title', 'artist']]
        .drop_duplicates()
        .reset_index(drop=True))
print('Всего уникальных треков:', len(uniq))
'''


"\n# Load credentials from .env securely\nfrom dotenv import load_dotenv\nimport os\nload_dotenv()\n\nimport spotipy\nfrom spotipy.oauth2 import SpotifyClientCredentials\n\n# Read credentials from environment (.env is gitignored)\nclient_id = os.environ.get('SPOTIFY_CLIENT_ID')\nclient_secret = os.environ.get('SPOTIFY_CLIENT_SECRET')\nif not client_id or not client_secret:\n    raise RuntimeError('SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET must be set in .env or environment')\n\n# Create auth manager using env credentials\nauth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)\nsp = spotipy.Spotify(auth_manager=auth_manager)\n\n# Далее работаем с файлами\nimport pandas as pd\n\ndf = pd.read_csv('songs_for_spotify.csv')  # файл с колонками title, artist\n\n# Подготовим поле для Spotify ID\ndf['spotify_id'] = None\n# df = объединённый Billboard+UK (или только UK) после чистки\nuniq = (df[['title', 'artist']]\n        .drop_duplicates()\n        .reset_

In [ ]:
import json, os

CACHE_PATH = "spotify_cache.json"
cache = {}
if os.path.exists(CACHE_PATH):
    cache = json.load(open(CACHE_PATH))

def save_cache():
    with open(CACHE_PATH, "w") as f:
        json.dump(cache, f)


In [ ]:
import yt_dlp
from youtubesearchpython import VideosSearch
import pandas as pd
import os
import time
import json
from urllib.parse import quote_plus

# Кэш для сохранения найденных ссылок
ID_CACHE_PATH = "id_cache.json"
id_cache = {}
if os.path.exists(ID_CACHE_PATH):
    with open(ID_CACHE_PATH, 'r') as f:
        id_cache = json.load(f)

def save_id_cache():
    with open(ID_CACHE_PATH, 'w') as f:
        json.dump(id_cache, f, indent=2)

def search_youtube_link(title, artist, max_retries=5):
    query = f"{artist} - {title}"
    
    # Проверяем кэш
    if query in id_cache:
        return id_cache[query]
    
    for attempt in range(max_retries):
        try:            
            # Увеличиваем timeout и добавляем больше времени между попытками
            videos_search = VideosSearch(
                query, 
                limit=1, 
                region='US', 
                timeout=60  # Увеличили timeout до 60 секунд
            )
            results = videos_search.result()
            
            if results['result']:
                url = results['result'][0]['link']
                # Сохраняем в кэш
                id_cache[query] = url
                save_id_cache()
                return url
            else:
                return None
                
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 10  # Увеличили задержку до 10, 20, 30, 40 секунд
                time.sleep(wait_time)
            else:
                # Сохраняем неудачную попытку в кэш
                id_cache[query] = None
                save_id_cache()
                return None

def download_track(youtube_url, title, artist, output_dir='.'):
    if youtube_url is None:
        return False
    
    # Безопасное имя файла
    safe_filename = f"{artist} - {title}".replace('/', '_').replace('\\', '_').replace(':', '_')
    
    # Проверяем, не существует ли уже файл
    mp3_path = f'{output_dir}/{safe_filename}.mp3'
    if os.path.exists(mp3_path):
        return True
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_dir}/{safe_filename}.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'postprocessor_args': [
            '-ss', '30',  # Начинаем с 30 секунды (пропускаем интро)
            '-t', '30'    # Берем 30 секунд
        ],
        'quiet': True,
        'ignoreerrors': True,
        'no_warnings': True,
        'socket_timeout': 60,  # Увеличиваем timeout для скачивания
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
            return True
    except Exception as e:
        return False

def process_csv(csv_file, output_dir='./tracks', start_from=0, limit=None, pause_between=2):
    df = pd.read_csv(csv_file)
    
    if limit:
        df = df.iloc[start_from:start_from + limit]
    else:
        df = df.iloc[start_from:]
    
    os.makedirs(output_dir, exist_ok=True)
    
    total = len(df)
    successful = 0
    failed = 0
    skipped = 0
    
    for idx, row in df.iterrows():
        title = str(row['title']).strip()
        artist = str(row['artist']).strip()
        
        # Проверяем, не существует ли уже файл
        safe_filename = f"{artist} - {title}".replace('/', '_').replace('\\', '_').replace(':', '_')
        mp3_path = f'{output_dir}/{safe_filename}.mp3'
        
        if os.path.exists(mp3_path):
            skipped += 1
            continue        
        youtube_url = search_youtube_link(title, artist)
        
        if youtube_url:
            if download_track(youtube_url, title, artist, output_dir):
                successful += 1
            else:
                failed += 1
        else:
            failed += 1
        
        # Увеличенная пауза между запросами
        time.sleep(pause_between)
        
        # Сохраняем прогресс каждые 50 треков
        if (idx + 1) % 50 == 0:
            print(f"\n--- Промежуточный отчет ---")
            print(f"Обработано: {idx + 1}/{total}")
            print(f"Успешно: {successful}, Неудачно: {failed}, Пропущено: {skipped}")
    
    print(f"\n=== ФИНАЛЬНЫЕ ИТОГИ ===")
    print(f"Всего обработано: {total}")
    print(f"Успешно скачано: {successful}")
    print(f"Неудачных попыток: {failed}")
    print(f"Пропущено (уже существовали): {skipped}")

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from tqdm import tqdm
import pandas as pd
import os

# Блокировки для безопасного доступа к кэшу
id_cache_lock = threading.Lock()
download_lock = threading.Lock()

def save_id_cache_thread_safe():
    with id_cache_lock:
        save_id_cache()

def search_youtube_link_thread_safe(title, artist, max_retries=5):
    query = f"{artist} - {title}"
    
    # Проверяем кэш с блокировкой
    with id_cache_lock:
        if query in id_cache:
            return id_cache[query]
    
    for attempt in range(max_retries):
        try:            
            videos_search = VideosSearch(
                query, 
                limit=1, 
                region='US', 
                timeout=60
            )
            results = videos_search.result()
            
            if results['result']:
                url = results['result'][0]['link']
                # Сохраняем в кэш с блокировкой
                with id_cache_lock:
                    id_cache[query] = url
                    save_id_cache()
                return url
            else:
                print(f"Результаты не найдены для: {query}")
                return None
                
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 5  # Уменьшили задержку
                print(f"Жду {wait_time} секунд перед следующей попыткой...")
                time.sleep(wait_time)
            else:
                print(f"Не удалось найти видео после {max_retries} попыток")
                # Сохраняем неудачную попытку в кэш
                with id_cache_lock:
                    id_cache[query] = None
                    save_id_cache()
                return None

def process_track(args):
    title, artist, output_dir = args
    
    # Безопасное имя файла
    safe_filename = f"{artist} - {title}".replace('/', '_').replace('\\', '_').replace(':', '_')
    mp3_path = f'{output_dir}/{safe_filename}.mp3'
    
    # Проверяем существование файла
    if os.path.exists(mp3_path):
        return "skipped", title, artist
    
    # Ищем видео на YouTube
    youtube_url = search_youtube_link_thread_safe(title, artist)
    if not youtube_url:
        return "failed", title, artist
    
    # Скачиваем трек (эта операция уже потокобезопасна в youtube-dl)
    if download_track(youtube_url, title, artist, output_dir):
        return "success", title, artist
    return "failed", title, artist


def process_csv_parallel(csv_file, output_dir='./tracks', start_from=0, limit=None, max_workers=3):
    df = pd.read_csv(csv_file)
    if limit:
        df = df.iloc[start_from:start_from + limit]
    else:
        df = df.iloc[start_from:]

    os.makedirs(output_dir, exist_ok=True)

    total = len(df)
    tqdm.write(f"Всего треков для обработки: {total}")

    # Подготавливаем аргументы для каждого трека
    track_args = [(row['title'].strip(), row['artist'].strip(), output_dir)
                  for _, row in df.iterrows()]

    successful = failed = skipped = 0

    # Прогресс-бар по количеству завершённых задач
    with ThreadPoolExecutor(max_workers=max_workers) as executor, \
         tqdm(total=total, desc="Скачивание треков", unit="trk") as pbar:

        # словарь future -> (title, artist), чтобы в логах писать имена
        future_map = {executor.submit(process_track, args): args for args in track_args}

        for future in as_completed(future_map):
            title, artist, _ = future_map[future]
            try:
                status, title, artist = future.result()
                if status == "success":
                    successful += 1
                elif status == "failed":
                    failed += 1
                    tqdm.write(f"❌ Ошибка:   {artist} - {title}")
                else:
                    skipped += 1
            except Exception as e:
                failed += 1
                tqdm.write(f"⚠️  Исключение на {artist} - {title}: {e}")

            # апдейтим прогресс и короткую сводку в хвосте бара
            pbar.update(1)
            pbar.set_postfix(success=successful, failed=failed, skipped=skipped)

    tqdm.write("\n=== ФИНАЛЬНЫЕ ИТОГИ ===")
    tqdm.write(f"Всего обработано: {total}")
    tqdm.write(f"Успешно скачано:  {successful}")
    tqdm.write(f"Неудачных попыток:{failed}")
    tqdm.write(f"Пропущено:        {skipped}")


## Загрузка аудио с YouTube

Для каждого трека из нашего датасета:
1. Ищем видео на YouTube
2. Скачиваем аудио дорожку
3. Извлекаем 30-секундный фрагмент
4. Сохраняем в формате MP3

Используется кэширование для:
- Сохранения найденных YouTube ссылок
- Избежания повторной загрузки существующих файлов
- Сохранения прогресса при прерывании

In [ ]:
'''
print("=== ГОТОВ К СКАЧИВАНИЮ ТРЕКОВ ===")
df_all = pd.read_csv("songs_for_spotify.csv")
print(f"Всего треков в базе: {len(df_all)}")
print(f"Уже скачано файлов в папке music: {len([f for f in os.listdir('./music') if f.endswith('.mp3')])}")

# Полная загрузка в 4 потока
process_csv_parallel("songs_for_spotify.csv", "./music", start_from=0, limit=None, max_workers=8)
'''

=== ГОТОВ К СКАЧИВАНИЮ ТРЕКОВ ===
Всего треков в базе: 75079
Уже скачано файлов в папке music: 68351
Всего треков для обработки: 75079


Скачивание треков:   1%|▏         | 952/75079 [00:02<01:10, 1045.28trk/s, failed=0, skipped=952, success=0]

[download]  67.6% of    2.54MiB at    1.34MiB/s ETA 00:00

ERROR: [youtube] h0U2QUGKbSE: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
Скачивание треков:   1%|▏         | 1096/75079 [00:13<25:52, 47.65trk/s, failed=0, skipped=1095, success=1]  

Скачивание треков:   3%|▎         | 1984/75079 [00:15<03:50, 317.09trk/s, failed=0, skipped=1974, success=10]

...

Скачивание треков: 100%|█████████▉| 75076/75079 [39:13<00:02,  1.12trk/s, failed=287, skipped=73104, success=1685]

Скачивание треков: 100%|█████████▉| 75077/75079 [39:14<00:02,  1.02s/trk, failed=287, skipped=73104, success=1686]

Скачивание треков: 100%|█████████▉| 75078/75079 [39:16<00:01,  1.17s/trk, failed=287, skipped=73104, success=1687]

Скачивание треков: 100%|██████████| 75079/75079 [39:27<00:00, 31.71trk/s, failed=287, skipped=73104, success=1688]


=== ФИНАЛЬНЫЕ ИТОГИ ===
Всего обработано: 75079
Успешно скачано:  1688
Неудачных попыток:287
Пропущено:        73104


Надо собрать фичи для анализа. Самые базовые - это темп и тональность.

In [8]:
df = pd.read_parquet("billboard_clean.parquet")
df.head


<bound method NDFrame.head of              date  rank                                title  \
0      2021-11-06     1                           Easy On Me   
1      2021-11-06     2                                 Stay   
2      2021-11-06     3                        Industry Baby   
3      2021-11-06     4                           Fancy Like   
4      2021-11-06     5                           Bad Habits   
...           ...   ...                                  ...   
330069 1958-08-04    96                        Over And Over   
330070 1958-08-04    97                     I Believe In You   
330071 1958-08-04    98                      Little Serenade   
330072 1958-08-04    99  I'll Get By (As Long As I Have You)   
330073 1958-08-04   100                                 Judy   

                               artist  last-week  peak_rank  weeks-on-board  \
0                               Adele        1.0          1               3   
1       The Kid LAROI & Justin Bieber      

In [ ]:
import librosa
import numpy as np
from tqdm import tqdm

# === настройки ===
MUSIC_DIR = "./music"
OUTPUT_CSV = "tracks_with_features_uk.csv"

# профили для тональности (Krumhansl)
MAJOR_PROFILE = np.array([6.35,2.23,3.48,2.33,4.38,4.09,2.52,5.19,2.39,3.66,2.29,2.88])
MINOR_PROFILE = np.array([6.33,2.68,3.52,5.38,2.60,3.53,2.54,4.75,3.98,2.69,3.34,3.17])
NOTES = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']


def estimate_key(y, sr):
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
    chroma = chroma / (chroma.sum(axis=0, keepdims=True) + 1e-9)
    pitch_profile = chroma.mean(axis=1)

    def best_match(profile):
        scores = []
        p = profile / profile.sum()
        for i in range(12):
            scores.append(np.corrcoef(p, np.roll(pitch_profile, i))[0,1])
        key_index = int(np.argmax(scores))
        return key_index, float(np.max(scores))

    maj_root, maj_score = best_match(MAJOR_PROFILE)
    min_root, min_score = best_match(MINOR_PROFILE)

    if maj_score >= min_score:
        return f"{NOTES[maj_root]} major"
    else:
        return f"{NOTES[min_root]} minor"

def process_track(filepath):
    try:
        y, sr = librosa.load(filepath, mono=True)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        key = estimate_key(y, sr)
        return tempo, key
    except Exception as e:
        print(f"[ERROR] {filepath}: {e}")
        return None, None


def main():
    df = pd.read_parquet("uk_clean.parquet")
    tempos, keys = [], []
    missed_songs = 0

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing tracks"):
        artist = row["artist"]
        title = row["title"]
        filename = os.path.join(MUSIC_DIR, f"{artist} - {title}.mp3")
        if os.path.exists(filename):
            tempo, key = process_track(filename)
        else:
            missed_songs += 1
            tempo, key = None, None

        tempos.append(tempo)
        keys.append(key)

    df["tempo"] = tempos
    df["key"] = keys

    df.to_csv(OUTPUT_CSV, index=False)
    print(f"Missed songs: {missed_songs}")
    print(f"Saved to {OUTPUT_CSV}")

main()


Processing tracks:   0%|          | 0/285245 [00:00<?, ?it/s]c:\Users\user\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
